### **STANDARDISING UNSTRUCTURED HEALTH DATA INTO FHIR R4**

In [1]:
import pandas as pd
import requests
import re
import json
import os
from IPython.display import display

In [5]:
file_path = r"C:\Users\Dell\Documents\UGM\PRPL\DataRaw1.csv" # Change with appropriate file path
df = pd.read_csv(file_path)

In [6]:
# Check for empty cells
empty_cells = df.isnull().sum()
print(empty_cells)

# Check for Duplicates
duplicates = df.duplicated().sum()
print("Number of duplicates:", duplicates)

Free_Text    0
dtype: int64
Number of duplicates: 0


In [7]:
from IPython.display import display

with pd.option_context('display.max_colwidth', None):
    display(df.head())

,Free_Text
0,"Pasien Rahmat Widodo, KTP 3276010101010004, 52 tahun, perempuan, alamat Jl. Imam Bonjol No. 78, Surabaya, nomor telepon 081987654321, golongan darah A, nomor rekam medis 727106, datang pada 10/09/2023. Diperiksa oleh Dr. Farida Lestari, KTP 3173010202020004. Keluhan: hipertensi, keluhan tambahan: sesak napas. disarankan mengubah pola makan. Dokter rujukan: Dr. Hasan. keluhan masih tetap dirasakan, perlu observasi lebih lanjut"
1,"Pasien Wawan Purnomo, KTP 3276010101010009, 62 tahun, perempuan, alamat Jl. Sudirman No. 56, Bandung, nomor telepon 081234567890, golongan darah B, nomor rekam medis 870412, datang pada 11/09/2023. Diperiksa oleh Dr. Andika Pratama, KTP 3173010202020003. Keluhan: migrain, keluhan tambahan: nyeri kepala. diberikan suplemen zat besi. Dokter rujukan: Dr. Hasan. direkomendasikan untuk menjaga pola makan dan olahraga"
2,"Pasien Lestari Nugroho, KTP 3276010101010007, 41 tahun, perempuan, alamat Jl. Imam Bonjol No. 78, Surabaya, nomor telepon 081987654321, golongan darah AB, nomor rekam medis 151697, datang pada 27/09/2023. Diperiksa oleh Dr. Sumarno Hadi, KTP 3173010202020007. Keluhan: hipertensi, keluhan tambahan: pusing. diberikan obat antihipertensi. Dokter rujukan: Dr. Hasan. direkomendasikan untuk menjaga pola makan dan olahraga"
3,"Pasien Yudi Raharjo, KTP 3276010101010004, 20 tahun, laki-laki, alamat Jl. Sudirman No. 56, Bandung, nomor telepon 081987654321, golongan darah A, nomor rekam medis 583165, datang pada 3/08/2023. Diperiksa oleh Dr. Wahyu Nugraha, KTP 3173010202020010. Keluhan: infeksi saluran pernapasan, keluhan tambahan: nyeri kepala. diberikan obat antihipertensi. Dokter rujukan: Dr. Dian. perlu dilakukan tes darah ulang"
4,"Pasien Wawan Purnomo, KTP 3276010101010004, 45 tahun, perempuan, alamat Jl. Diponegoro No. 12, Medan, nomor telepon 081234567890, golongan darah A, nomor rekam medis 522131, datang pada 25/08/2023. Diperiksa oleh Dr. Sari Utami, KTP 3173010202020009. Keluhan: migrain, keluhan tambahan: sakit pinggang. diberikan antibiotik selama 5 hari. Dokter rujukan: Dr. Dian. diminta mengurangi konsumsi garam dan lemak"


### **SIMPLE ENTITY EXTRACTION**

In [9]:
#SIMPLE ENTITY EXTRACTION
def extract_simple(text):
    # KTP (GIVEN THAT NIK IS ALWAYS 16 DIGITS LONG)
    ktp_matches = re.findall(r'KTP (\d{16})', text)
    patient_ktp = ktp_matches[0] if len(ktp_matches) > 0 else None
    doctor_ktp = ktp_matches[1] if len(ktp_matches) > 1 else None
    
    # AGE
    age_match = re.search(r'(\d+)\s*tahun', text)
    age = int(age_match.group(1)) if age_match else None
    
    # GENDER
    gender_match = re.search(r'(laki-laki|perempuan)', text, re.IGNORECASE)
    gender = gender_match.group(1).lower() if gender_match else None
    
    # PHONE (GIVEN THAT ALL THE PHON NUMBERS ARE INDONESIAN AND STARTS WITH "08XXXX")
    phone_match = re.search(r'nomor telepon\s+(08\d{8,11})', text)
    phone = phone_match.group(1) if phone_match else None
    
    # ASSUMING THE MEDICAL RECORD NO IS ALWAYS 6 DIGITS LONG
    rekam_medis_match = re.search(r'\b(\d{6})\b', text)
    rekam_medis = rekam_medis_match.group(1) if rekam_medis_match else None
    
    # BLOOD TYPE
    blood_type_match = re.search(r'\b([ABO]|AB)[+-]?\b', text)
    blood_type = blood_type_match.group(0) if blood_type_match else None
    
    # PRIMARY AND ADDITIONAL COMPLAINTS
    complaint_match = re.search(r'keluhan\s*:\s*([^,.]+)', text, re.IGNORECASE)
    additional_complaint_match = re.search(r'keluhan\s*tambahan\s*:\s*([^,.]+)', text, re.IGNORECASE)

    primary_complaint = [complaint_match.group(1).strip()] if complaint_match else []
    additional_complaint = [additional_complaint_match.group(1).strip()] if additional_complaint_match else []

    # PRIMARY AND ADDITIONAL RECOMMENDATIONS & NOTES
    recommendation_matches = re.findall(r'(diberikan|disarankan|direkomendasikan)\s([^,.]+)', text, re.IGNORECASE)
    recommendation = [f"{match[0]} {match[1].strip()}" for match in recommendation_matches]

    notes_match = re.search(r'Dokter rujukan\s*:\s*[^.]+\.?\s*(.*)', text, re.IGNORECASE)
    notes = notes_match.group(1).strip() if notes_match else ""
    notes = re.sub(r'^[A-Za-z]+\b', '', notes).strip()
    notes = notes.split('. ') if notes else []
    notes = [note.strip() for note in notes if note.strip()]  # Clean up and remove empty strings

    # REMOVES DUPLICATE NOTES AND RECOMMENDATIONS
    recommendation = list(set(recommendation))  # Remove duplicates within recommendations
    notes = list(set(notes))  # Remove duplicates within notes

    # Ensure recommendations don't overlap with notes
    filtered_recommendation = []
    for rec in recommendation:
        if rec in notes:
            continue  # If the recommendation already exists in notes, exclude it from recommendations
        filtered_recommendation.append(rec)

    # Finalize recommendations and notes
    recommendation = filtered_recommendation

    return {
        "PATIENT_KTP": patient_ktp,
        "DOCTOR_KTP": doctor_ktp,
        "AGE": age,
        "GENDER": gender,
        "PHONE": phone,
        "MEDICAL_RECORD_NUMBER": rekam_medis,
        "BLOOD_TYPE": blood_type,
        "MAIN_COMPLAINT": primary_complaint,
        "ADDITIONAL_COMPLAINT": additional_complaint,
        "RECOMMENDATION": recommendation,
        "NOTES": notes
    }

In [10]:
# APPLY ON DATASET

df['Simple_Fields'] = df['Free_Text'].apply(extract_simple)
df_SimpleFields = pd.json_normalize(df['Simple_Fields'])
df = pd.concat([df, df_SimpleFields], axis=1)

In [11]:
df_SimpleFields.head()

,PATIENT_KTP,DOCTOR_KTP,AGE,GENDER,PHONE,MEDICAL_RECORD_NUMBER,BLOOD_TYPE,MAIN_COMPLAINT,ADDITIONAL_COMPLAINT,RECOMMENDATION,NOTES
0,3276010101010004,3173010202020004,52,perempuan,081987654321,727106,A,[hipertensi],[sesak napas],[disarankan mengubah pola makan],"[keluhan masih tetap dirasakan, perlu observas..."
1,3276010101010009,3173010202020003,62,perempuan,081234567890,870412,B,[migrain],[nyeri kepala],[diberikan suplemen zat besi],[direkomendasikan untuk menjaga pola makan dan...
2,3276010101010007,3173010202020007,41,perempuan,081987654321,151697,AB,[hipertensi],[pusing],[diberikan obat antihipertensi],[direkomendasikan untuk menjaga pola makan dan...
3,3276010101010004,3173010202020010,20,laki-laki,081987654321,583165,A,[infeksi saluran pernapasan],[nyeri kepala],[diberikan obat antihipertensi],[perlu dilakukan tes darah ulang]
4,3276010101010004,3173010202020009,45,perempuan,081234567890,522131,A,[migrain],[sakit pinggang],[diberikan antibiotik selama 5 hari],[diminta mengurangi konsumsi garam dan lemak]


### **COMPLEX ENTITIES EXTRACTION**

In [12]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# USING AN INDOBERT MODEL THAT HAS BEEN SPECIFICALLLY TRAINED FOR NER TASKS

tokenizer = AutoTokenizer.from_pretrained("syafiqfaray/indobert-model-ner")
model = AutoModelForTokenClassification.from_pretrained("syafiqfaray/indobert-model-ner")

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\pipelines\token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [14]:
# COMPLEX ENTITIES EXTRACTIONS
def extract_complex(text):
    ner_results = ner_pipeline(text)
    extracted_info = {
        "PATIENT_NAME": None,
        "DOCTOR_NAME": None,
        "REFERRAL": None,
        "ADDRESS": [],
        "CITY": None,
        "DATE_VISIT": []
    }

    unclassified_names = []

    for entity in ner_results:
        label = entity.get('entity_group', 'Unknown')
        word = entity['word'].strip()

        if label == "PER":                      # NAMES
            unclassified_names.append(word)
        elif label == "LOC":                    # ADDRESS
            extracted_info['ADDRESS'].append(word)
        elif label == "GPE":                    # CITY
            extracted_info['CITY'] = word 
        elif label == "DAT":                    # DATE
            extracted_info['DATE_VISIT'].append(word)

    # Classify names based on order or context
    if unclassified_names:
        if len(unclassified_names) >= 1:
            extracted_info['PATIENT_NAME'] = unclassified_names[0]  # Assume first name is patient
        if len(unclassified_names) >= 2:
            extracted_info['DOCTOR_NAME'] = unclassified_names[1]  # Assume second name is doctor
        if len(unclassified_names) >= 3:
            extracted_info['REFERRAL'] = unclassified_names[2]  # Assume third name is referring doctor
            
    # ADDRESS & CITY
    if extracted_info['ADDRESS']:  # Combine address tokens into a single string
        extracted_info['ADDRESS'] = ' '.join(extracted_info['ADDRESS']).strip()
    else:
        extracted_info['ADDRESS'] = None

    if not extracted_info['CITY']:  # If city is not detected, set it to None
        extracted_info['CITY'] = None

    # Clean and finalize extracted information
    for key in extracted_info:
        if isinstance(extracted_info[key], list) and extracted_info[key]:  # Clean list entries
            extracted_info[key] = ' '.join(extracted_info[key]).strip()
        elif not extracted_info[key]:  # None if empty
            extracted_info[key] = None
    
    return extracted_info

In [15]:
# APPLY ON DATASET

df['Complex_Fields'] = df['Free_Text'].apply(extract_complex)
df_ComplexFields = pd.json_normalize(df['Complex_Fields'])
data = pd.concat([df, df_ComplexFields], axis=1)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [16]:
df_ComplexFields.head()

,PATIENT_NAME,DOCTOR_NAME,REFERRAL,ADDRESS,CITY,DATE_VISIT
0,rahmat widodo,dr. farida lestari,dr. hasan,jl. imam bonjol no. 78,surabaya,10 / 09 / 2023
1,wawan purnomo,dr. andika pratama,dr. hasan,jl. sudirman no. 56,bandung,11 / 09 / 2023
2,lestari nugroho,dr. sumarno hadi,dr. hasan,jl. imam bonjol no. 78,surabaya,27 / 09 / 2023
3,yudi raharjo,dr. wahyu nugraha,dr. dian,jl. sudirman no. 56,bandung,3 / 08 / 2023
4,wawan purnomo,dr. sari utami,dr. dian,jl. diponegoro no. 12,medan,25 / 08 / 2023


### **EXTRACTED INFORMATION STANDARDIZATION & NORMALIZATION**

In [17]:
simple_expand = pd.json_normalize(df['Simple_Fields'])
complex_expand = pd.json_normalize(df['Complex_Fields'])

Extracted_Info = pd.concat([simple_expand, complex_expand], axis=1)

In [18]:
from IPython.display import display

with pd.option_context('display.max_colwidth', None):
    display(Extracted_Info.head())

,PATIENT_KTP,DOCTOR_KTP,AGE,GENDER,PHONE,MEDICAL_RECORD_NUMBER,BLOOD_TYPE,MAIN_COMPLAINT,ADDITIONAL_COMPLAINT,RECOMMENDATION,NOTES,PATIENT_NAME,DOCTOR_NAME,REFERRAL,ADDRESS,CITY,DATE_VISIT
0,3276010101010004,3173010202020004,52,perempuan,081987654321,727106,A,[hipertensi],[sesak napas],[disarankan mengubah pola makan],"[keluhan masih tetap dirasakan, perlu observasi lebih lanjut]",rahmat widodo,dr. farida lestari,dr. hasan,jl. imam bonjol no. 78,surabaya,10 / 09 / 2023
1,3276010101010009,3173010202020003,62,perempuan,081234567890,870412,B,[migrain],[nyeri kepala],[diberikan suplemen zat besi],[direkomendasikan untuk menjaga pola makan dan olahraga],wawan purnomo,dr. andika pratama,dr. hasan,jl. sudirman no. 56,bandung,11 / 09 / 2023
2,3276010101010007,3173010202020007,41,perempuan,081987654321,151697,AB,[hipertensi],[pusing],[diberikan obat antihipertensi],[direkomendasikan untuk menjaga pola makan dan olahraga],lestari nugroho,dr. sumarno hadi,dr. hasan,jl. imam bonjol no. 78,surabaya,27 / 09 / 2023
3,3276010101010004,3173010202020010,20,laki-laki,081987654321,583165,A,[infeksi saluran pernapasan],[nyeri kepala],[diberikan obat antihipertensi],[perlu dilakukan tes darah ulang],yudi raharjo,dr. wahyu nugraha,dr. dian,jl. sudirman no. 56,bandung,3 / 08 / 2023
4,3276010101010004,3173010202020009,45,perempuan,081234567890,522131,A,[migrain],[sakit pinggang],[diberikan antibiotik selama 5 hari],[diminta mengurangi konsumsi garam dan lemak],wawan purnomo,dr. sari utami,dr. dian,jl. diponegoro no. 12,medan,25 / 08 / 2023


In [19]:
# STANDARDIZING & CHANGING NECESSARY COLUMNS TO SATUSEHAT AND FHIR4 COMPLIANT FORMAT

# FHIR as well as SATUSEHAT standards requires the gender information to be in English
Extracted_Info['GENDER'] = Extracted_Info['GENDER'].replace({"perempuan": "female", "laki-laki": "male"})

# Assuming that all of the phone numbers are Indonesian, we can normalize it to fit International standards 
Extracted_Info['PHONE'] = Extracted_Info['PHONE'].apply(lambda x: "+62" + x[1:] if x.startswith("0") else x)

# To get the year the patient visited to find their birth year later on
Extracted_Info["DATE_VISIT"] = pd.to_datetime(Extracted_Info["DATE_VISIT"], format="%d / %m / %Y", dayfirst=True).dt.strftime("%Y-%m-%d")

In [20]:
Extracted_Info.head()

,PATIENT_KTP,DOCTOR_KTP,AGE,GENDER,PHONE,MEDICAL_RECORD_NUMBER,BLOOD_TYPE,MAIN_COMPLAINT,ADDITIONAL_COMPLAINT,RECOMMENDATION,NOTES,PATIENT_NAME,DOCTOR_NAME,REFERRAL,ADDRESS,CITY,DATE_VISIT
0,3276010101010004,3173010202020004,52,female,+6281987654321,727106,A,[hipertensi],[sesak napas],[disarankan mengubah pola makan],"[keluhan masih tetap dirasakan, perlu observas...",rahmat widodo,dr. farida lestari,dr. hasan,jl. imam bonjol no. 78,surabaya,2023-09-10
1,3276010101010009,3173010202020003,62,female,+6281234567890,870412,B,[migrain],[nyeri kepala],[diberikan suplemen zat besi],[direkomendasikan untuk menjaga pola makan dan...,wawan purnomo,dr. andika pratama,dr. hasan,jl. sudirman no. 56,bandung,2023-09-11
2,3276010101010007,3173010202020007,41,female,+6281987654321,151697,AB,[hipertensi],[pusing],[diberikan obat antihipertensi],[direkomendasikan untuk menjaga pola makan dan...,lestari nugroho,dr. sumarno hadi,dr. hasan,jl. imam bonjol no. 78,surabaya,2023-09-27
3,3276010101010004,3173010202020010,20,male,+6281987654321,583165,A,[infeksi saluran pernapasan],[nyeri kepala],[diberikan obat antihipertensi],[perlu dilakukan tes darah ulang],yudi raharjo,dr. wahyu nugraha,dr. dian,jl. sudirman no. 56,bandung,2023-08-03
4,3276010101010004,3173010202020009,45,female,+6281234567890,522131,A,[migrain],[sakit pinggang],[diberikan antibiotik selama 5 hari],[diminta mengurangi konsumsi garam dan lemak],wawan purnomo,dr. sari utami,dr. dian,jl. diponegoro no. 12,medan,2023-08-25


In [39]:
# TO FIND PATIENT'S BIRTH YEAR BASED ON THEIR AGE AND YEAR OF VIIST

# FHIR requires the birthdate and does not accept age (birthdate does not have to be complete - it accepts only birth year)
Extracted_Info["VISIT_YEAR"] = pd.to_datetime(Extracted_Info["DATE_VISIT"]).dt.year
Extracted_Info["BIRTH_YEAR"] = Extracted_Info["VISIT_YEAR"] - Extracted_Info["AGE"]

In [40]:
# VISUALIZATION PURPOSES
ColOrder = ["PATIENT_NAME", "PATIENT_KTP", "MEDICAL_RECORD_NUMBER", "BIRTH_YEAR", "GENDER", "ADDRESS", "CITY", "PHONE",
            "DOCTOR_NAME", "DOCTOR_KTP", "DATE_VISIT", "MAIN_COMPLAINT", "ADDITIONAL_COMPLAINT", "RECOMMENDATION",
            "REFERRAL", "NOTES"]

Extracted_Clean = Extracted_Info[ColOrder] # BLOOD TYPE IS LEFT OUT SINCE IT IS NOT NECESSARY IN FHIR R4 FORMAT

In [41]:
Extracted_Clean.head()

,PATIENT_NAME,PATIENT_KTP,MEDICAL_RECORD_NUMBER,BIRTH_YEAR,GENDER,ADDRESS,CITY,PHONE,DOCTOR_NAME,DOCTOR_KTP,DATE_VISIT,MAIN_COMPLAINT,ADDITIONAL_COMPLAINT,RECOMMENDATION,REFERRAL,NOTES
0,rahmat widodo,3276010101010004,727106,1971,female,jl. imam bonjol no. 78,surabaya,+6281987654321,dr. farida lestari,3173010202020004,2023-09-10,[hipertensi],[sesak napas],[disarankan mengubah pola makan],dr. hasan,"[keluhan masih tetap dirasakan, perlu observas..."
1,wawan purnomo,3276010101010009,870412,1961,female,jl. sudirman no. 56,bandung,+6281234567890,dr. andika pratama,3173010202020003,2023-09-11,[migrain],[nyeri kepala],[diberikan suplemen zat besi],dr. hasan,[direkomendasikan untuk menjaga pola makan dan...
2,lestari nugroho,3276010101010007,151697,1982,female,jl. imam bonjol no. 78,surabaya,+6281987654321,dr. sumarno hadi,3173010202020007,2023-09-27,[hipertensi],[pusing],[diberikan obat antihipertensi],dr. hasan,[direkomendasikan untuk menjaga pola makan dan...
3,yudi raharjo,3276010101010004,583165,2003,male,jl. sudirman no. 56,bandung,+6281987654321,dr. wahyu nugraha,3173010202020010,2023-08-03,[infeksi saluran pernapasan],[nyeri kepala],[diberikan obat antihipertensi],dr. dian,[perlu dilakukan tes darah ulang]
4,wawan purnomo,3276010101010004,522131,1978,female,jl. diponegoro no. 12,medan,+6281234567890,dr. sari utami,3173010202020009,2023-08-25,[migrain],[sakit pinggang],[diberikan antibiotik selama 5 hari],dr. dian,[diminta mengurangi konsumsi garam dan lemak]


### **JSON MAPPING**

#### **PATIENT RESOURCE**

In [26]:
def map_patient(row, patient_ID):
    return {
     "resourceType": "Patient",
     "id": patient_ID,
     "identifier": [
         {
             "system": "http://satusehat.kemkes.go.id/nik", # Patient KTP
             "value": row["PATIENT_KTP"]
         },
         {
             "system": "https://fhir.kemkes.go.id/id/ihs-number", # Medical Record Number
             "value": row["MEDICAL_RECORD_NUMBER"]
         }, 
     ],
     "active": True,
     "name": [
         {
             "use": "official",
             "text": row["PATIENT_NAME"]
         }
     ],
     "gender": row["GENDER"],
     "birthDate": str(row["BIRTH_YEAR"]),
     "telecom": [
         {
             "system": "phone",
             "value": row["PHONE"]
         }
     ],
     "address": [
         {
             "line": [row["ADDRESS"]],
             "city": row["CITY"],
             "country": "ID"
         }
     ],
     "meta": {
         "profile": [
             "http://hl7.org/fhir/StructuredDefinition/Patient" # DUMMY LINK
         ]
     }
    }

#### **PRACTITIONER RESOURCE**

In [28]:
def map_practitioner(row, practitioner_ID):
    return {
        "resourceType": "Practitioner",
        "id": practitioner_ID,
        "identifier": [
            {
                "system": "http://satusehat.kemkes.go.id/nik", # Doctor KTP
                "value": row["DOCTOR_KTP"]
            }
        ],
        "active": True,
        "name": [
            {
                "text": row["DOCTOR_NAME"]
            }
        ],
        "meta": {
            "profile": [
                "http://hl7.org/fhir/StructuredDefinition/Practitioner" # DUMMY LINK
            ]
        }
    }

#### **CONDITION RESOURCE**

In [29]:
def map_condition(row, condition_ID):
    return {
        "resourceType": "Condition",
        "id": condition_ID,
        "clinicalStatus": {
            "coding": [
                {
                    "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
                    "code": "active",
                    "display": "Active"
                }
            ]
        },
        
        # Patient complaints
        "code": {
            "text": ", ".join(row.get("MAIN_COMPLAINT", []) + row.get("ADDITIONAL_COMPLAINT", []))
        },
        "note": {
            "text": row["RECOMMENDATION"]  
        },
        "subject": {
            "reference": f"Patient/{row['PATIENT_KTP']}"
        },
        "onsetDateTime": row["DATE_VISIT"]
    }

#### **ENCOUNTER RESOURCE**

In [30]:
def map_encounter(row, encounter_ID):
    return{
        "resourceType": "Encounter",
        "id": encounter_ID,
        "subject": {
            "reference": f"Patient/{row['PATIENT_KTP']}"
        },
        "period": {
            "start": row["DATE_VISIT"]
        },
        # Referral doctor
        "participant": [
            {
                "individual": {
                    "reference": f"Referral Doctor/{row['REFERRAL']}"
                }
            }
        ],
        "note": [
            {
                "text": row["NOTES"]
            }
        ]
    }

#### **COMBINING RESOURCES INTO ONE JSON STRUCTURE**

In [31]:
import uuid

def generate_uuid():
    return str(uuid.uuid4())

def map_all(row):
    # Generate a consistent ID for the patient
    patient_id = generate_uuid()
    practitioner_id = patient_id
    encounter_id = patient_id
    condition_id = patient_id
    
    return {
        "Patient": map_patient(row, patient_id),
        "Practitioner": map_practitioner(row, practitioner_id),
        "Condition": map_condition(row, condition_id),
        "Encounter": map_encounter(row, encounter_id)
    }

In [32]:
# SAVING ALL ROWS INTO INDIVIDUAL .JSON FILES
import os
import json

def saveJSON (df, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    for index, row in df.iterrows():
        row_data = map_all(row)
        
        output_file = os.path.join(output_dir, f"row_{index+1}.json")
        
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(row_data, f, indent=4, ensure_ascii=False)
            
        print(f"Saved JSON for row {index+1} to {output_file} successfully.")

In [61]:
# SAVING THE FILES LOCALLY

output_dir = r"C:\Users\Dell\Documents\UGM\PRPL\OUTPUT2" # Change with appropriate file path
saveJSON(Extracted_Clean, output_dir)

Saved JSON for row 1 to C:\Users\Dell\Documents\UGM\PRPL\OUTPUT2\row_1.json successfully.
Saved JSON for row 2 to C:\Users\Dell\Documents\UGM\PRPL\OUTPUT2\row_2.json successfully.
Saved JSON for row 3 to C:\Users\Dell\Documents\UGM\PRPL\OUTPUT2\row_3.json successfully.
Saved JSON for row 4 to C:\Users\Dell\Documents\UGM\PRPL\OUTPUT2\row_4.json successfully.
Saved JSON for row 5 to C:\Users\Dell\Documents\UGM\PRPL\OUTPUT2\row_5.json successfully.
Saved JSON for row 6 to C:\Users\Dell\Documents\UGM\PRPL\OUTPUT2\row_6.json successfully.
Saved JSON for row 7 to C:\Users\Dell\Documents\UGM\PRPL\OUTPUT2\row_7.json successfully.
Saved JSON for row 8 to C:\Users\Dell\Documents\UGM\PRPL\OUTPUT2\row_8.json successfully.
Saved JSON for row 9 to C:\Users\Dell\Documents\UGM\PRPL\OUTPUT2\row_9.json successfully.
Saved JSON for row 10 to C:\Users\Dell\Documents\UGM\PRPL\OUTPUT2\row_10.json successfully.
Saved JSON for row 11 to C:\Users\Dell\Documents\UGM\PRPL\OUTPUT2\row_11.json successfully.
Saved 

### **FREE TEXT USER INPUT**

In [63]:
def free_input1():
    print("Enter your text: ")
    text = input("> ").strip()
    
    print("\nInput Text:")
    print(text)
    
    simple_data = extract_simple(text)
    complex_data = extract_complex(text)
    combined_data = {**simple_data, **complex_data}
    
    df_Free = pd.DataFrame([combined_data])
    
    df_Free['GENDER'] = df_Free['GENDER'].replace({"perempuan": "female", "laki-laki": "male"})
    df_Free['PHONE'] = df_Free['PHONE'].apply(lambda x: "+62" + x[1:] if x.startswith("0") else x)
    df_Free['DATE_VISIT'] = pd.to_datetime(df_Free['DATE_VISIT'], format="%d / %m / %Y", dayfirst=True).dt.strftime("%Y-%m-%d")
    df_Free['VISIT_YEAR'] = pd.to_datetime(df_Free['DATE_VISIT']).dt.year
    AssumedYear = 2024
    df_Free['BIRTH_YEAR'] = AssumedYear - df_Free['AGE']
    
    standardized_data = df_Free.to_dict(orient="records")[0]
    
    # Display the results
    print("\nStandardized Information:")
    for key, value in standardized_data.items():
        print(f"{key}: {value}")
        
    # Apply `map_all` to the DataFrame row
    mapped_data = map_all(df_Free.iloc[0])  # Pass the first (and only) row as a Series to `map_all`
    
    # DEBUGGING PURPOSES - SHOW FHIR FORMAT
    print("\nMapped FHIR Resources:")
    for resource_type, resource_data in mapped_data.items():
        print(f"\n{resource_type}:")
        print(json.dumps(resource_data, indent=4))
    
    # Save to JSON if desired
    save_choice = input("\nDo you want to save the standardized data to a JSON file? (yes/no): ").strip().lower()
    if save_choice == 'yes':
        output_file = f"JSON_FORMAT_{generate_uuid()}.json"
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(mapped_data, f, indent=4, ensure_ascii=False)
        print(f"File saved to {output_file}")

In [64]:
# Define the main function
def main1():
    print("Type 'exit' at any time to quit.\n")
    
    while True:
        user_input = input("Do you want to process a new text? (yes/no): ").strip().lower()
        if user_input == "yes":
            free_input()
        elif user_input == "no" or user_input == "exit":
            print("Exit")
            break
        else:
            print("Invalid input. Please type 'yes' or 'no'.")

In [ ]:
# Run the program
if __name__ == "__main__":
    main1()

### **USER INPUT (DEMO PURPOSES VISUALIZATION)**

In [56]:
def process_input():
    def on_submit():
        text = input_text.get("1.0", tk.END).strip()
        if not text:
            messagebox.showerror("Error", "Input text cannot be empty.")
            return

        simple_data = extract_simple(text)
        complex_data = extract_complex(text)
        combined_data = {**simple_data, **complex_data}

        df_Free = pd.DataFrame([combined_data])

        df_Free['GENDER'] = df_Free['GENDER'].replace({"perempuan": "female", "laki-laki": "male"})
        df_Free['PHONE'] = df_Free['PHONE'].apply(lambda x: "+62" + x[1:] if x.startswith("0") else x)
        df_Free['DATE_VISIT'] = pd.to_datetime(df_Free['DATE_VISIT'], format="%d / %m / %Y", dayfirst=True).dt.strftime("%Y-%m-%d")
        df_Free['VISIT_YEAR'] = pd.to_datetime(df_Free['DATE_VISIT']).dt.year
        df_Free['BIRTH_YEAR'] = df_Free['VISIT_YEAR'] - df_Free['AGE']

        standardized_data = df_Free.to_dict(orient="records")[0]

        mapped_data = map_all(df_Free.iloc[0])

        result_text = "Standardized Information:\n" + "\n".join(f"{k}: {v}" for k, v in standardized_data.items())
        result_text += "\n\nMapped FHIR Resources:\n"
        for resource_type, resource_data in mapped_data.items():
            result_text += f"\n{resource_type}:\n{json.dumps(resource_data, indent=4)}\n"

        output_text.config(state=tk.NORMAL)
        output_text.delete("1.0", tk.END)
        output_text.insert(tk.END, result_text)
        output_text.config(state=tk.DISABLED)

        save_button.config(state=tk.NORMAL)

        def save_to_file():
            output_file = filedialog.asksaveasfilename(defaultextension=".json", filetypes=[("JSON files", "*.json")])
            if output_file:
                with open(output_file, 'w', encoding='utf-8') as f:
                    json.dump(mapped_data, f, indent=4, ensure_ascii=False)
                messagebox.showinfo("Success", f"File saved to {output_file}")

        save_button.config(command=save_to_file)

    # Create the window
    window = tk.Tk()
    window.title("PRPL Demo")

    tk.Label(window, text="Enter your free text medical notes:").pack(pady=5)

    input_text = tk.Text(window, height=10, width=50)
    input_text.pack(pady=5)

    submit_button = tk.Button(window, text="Process", command=on_submit)
    submit_button.pack(pady=5)

    tk.Label(window, text="Output:").pack(pady=5)

    output_text = tk.Text(window, height=15, width=50, state=tk.DISABLED)
    output_text.pack(pady=5)

    save_button = tk.Button(window, text="Save to JSON", state=tk.DISABLED)
    save_button.pack(pady=5)

    window.mainloop()

In [57]:
def main():
    process_input()

In [62]:
# Run the program
if __name__ == "__main__":
    main()